<a href="https://colab.research.google.com/github/NPCA-TEAM/COVID-19/blob/main/Scripts/%202%20-%20INTEGRAR_DADOS_covid_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação e importação de bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install xlsxwriter

In [ ]:
import os
import glob
import pandas as pd # dataframes
import numpy as nd
import matplotlib.pyplot as plt # plotagem dos dados
import os
import time
from datetime import datetime, timedelta, date

# **CASOS**

#Leitura e visualização dos dados

##Casos média móvel Atual, 7 Dias, 14 Dias

In [ ]:
#Seleciona os arquivos de CASOS recentemente baixado da pasta de .CSVs (LakeData)
pasta = '/content/drive/MyDrive/NPCA - COVID/LakeData/Casos/' 
lista_de_arquivos_casos = glob.glob(pasta+'*') 
print(*lista_de_arquivos_casos, sep = '\n')
print('-----------------------------------------------------------------------------')

In [ ]:
#Indice dos arquivos na lista ordenada. ALTERAR NUMEROS PARA 7 E 14 QUANDO TIVERMOS AO MENOS 15 ARQUIVOS NA PASTA
indice_Setimo_dia_casos = 7
indice_DecimoQuarto_dia_casos = 14

#Ordena arquivos por data de criação do arquivo. Ultimo arquivo a ser criado começa no inicio da lista.
lista_de_arquivos_ordenados_casos= sorted(lista_de_arquivos_casos,key=os.path.getctime,reverse=True)
print(*lista_de_arquivos_ordenados_casos,sep = "\n")
print('---------------------------------------------------------------------------')

In [ ]:
#Seleciona o ultimo arquivo que foi criado e adicioado na pasta
arquivo_atual_casos = lista_de_arquivos_ordenados_casos[0]
#Seleciona o setimo arquivo que foi criado depois do ultimo arquivo adicionado na pasta
arquivo_7dias_casos = lista_de_arquivos_ordenados_casos[indice_Setimo_dia_casos]
#Seleciona o decimo quarto arquivo que foi criado depois do ultimo arquivo adicionado na pasta
arquivo_14dias_casos = lista_de_arquivos_ordenados_casos[indice_DecimoQuarto_dia_casos]

print('--------------------------------------------------------------------------------')
print(arquivo_atual_casos)
print(arquivo_7dias_casos)
print(arquivo_14dias_casos)

In [ ]:
#Importando arquivos
casos_media_movel_atual = pd.read_csv(arquivo_atual_casos, parse_dates=['data'])
casos_media_movel_atual.info()

casos_media_movel_7dias = pd.read_csv(arquivo_7dias_casos, parse_dates=['data'])
casos_media_movel_7dias.info()

casos_media_movel_14dias = pd.read_csv(arquivo_14dias_casos, parse_dates=['data'])
casos_media_movel_14dias.info()

#Porcentagem de vacinação: primeira, segunda e terceira dose.

In [ ]:
# importando os dados da COVID diretamente do GitHub
dados = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")

# alterar a coluna de data para o formato de data e hora
dados['date'] = pd.to_datetime(dados['date'])

# Filtrando os dados do Braisl, e mostrando apenas as colunas especificas:
dados_PA_vacinados = dados.query("state == 'PA'")[['date', 
                                                   'vaccinated_per_100_inhabitants',
                                                   'vaccinated_second_per_100_inhabitants',
                                                   'vaccinated_third_per_100_inhabitants']]

dados_PA_vacinados.columns=['data','Vacinacao_Dose1_PA','Vacinacao_Dose2_PA','Vacinacao_Dose3_PA']

plt.figure(figsize=(16, 4))
plt.grid()
plt.xlabel('date')
plt.ylabel('doses')
plt.title('COVID-19 Vaccination graph in Pará')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose1_PA'], label='Vaccination_dose1_PA')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose2_PA'], label='Vaccination_dose2_PA')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose3_PA'], label='Vaccination_dose3_PA')
plt.legend()

# Salvando o gráfico em PNG:
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Vaccination doses in Pará - Data collected day {date.today()}.png', dpi=300, transparent=True, bbox_inches='tight')

# Imprimindo o dataframe:
print(dados_PA_vacinados)

# Tratamento dos dados

##Tratamento para casos média móvel 

In [ ]:
#     PARA O DIA ATUAL - CASOS

# Filtrando apenas a data e retirando o horario:
casos_media_movel_atual['data'] = pd.to_datetime(casos_media_movel_atual['data'].dt.strftime('%Y-%m-%d'))

# Tornando Data em indice:
casos_media_movel_atual = casos_media_movel_atual.set_index('data')

#Alterando nome das colunas
casos_media_movel_atual = casos_media_movel_atual.rename(columns={'Qtd. pela data da publicação':'Casos_Publicacao_MM_atual_PA','Qtd. pela data do sintoma':'Casos_DataSintoma_MM_atual_PA'})                                                            
                                
# Plotando os dados:
strDiaColetado = str(casos_media_movel_atual.index[-1])
strDiaColetado = strDiaColetado.split(" ")[0]

# Plotando os dados:
plt.figure(figsize=(16, 3))
plt.grid()
plt.xlabel('date')
plt.ylabel('doses')

plt.title(f'Moving average of COVID-19 cases in Pará')
plt.plot(casos_media_movel_atual['Casos_DataSintoma_MM_atual_PA'], label='Cases_DateSymptom_MM_Current_PA')
plt.plot(casos_media_movel_atual['Casos_Publicacao_MM_atual_PA'], label='Cases_Publication_MM_Current_PA')
plt.legend()

print(casos_media_movel_atual) 

# Salvando o frafico em PNG:
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Moving average of cases in Pará - Data collected day {strDiaColetado}.png', dpi=300, transparent=True, bbox_inches='tight')     

In [ ]:
#      PARA 7 DIAS - CASOS

# Filtrando apenas a data e retirando o horario
casos_media_movel_7dias['data'] = pd.to_datetime(casos_media_movel_7dias['data'].dt.strftime('%Y-%m-%d'))

# Tornando Data em indice:
casos_media_movel_7dias = casos_media_movel_7dias.set_index('data')

#Alterando nome das colunas
casos_media_movel_7dias = casos_media_movel_7dias.rename(columns={'Qtd. pela data da publicação':'Casos_Publicacao_MM_7dias_PA','Qtd. pela data do sintoma':'Casos_DataSintoma_MM_7dias_PA'})
                                                                                                                                                                                                                                                                                         

# Plotando os dados:
strDiaColetado = str(casos_media_movel_7dias.index[-1])
strDiaColetado = strDiaColetado.split(" ")[0]

# Plotando os dados:
plt.figure(figsize=(16, 3))
plt.grid()
plt.xlabel('date')
plt.ylabel('cases')

plt.title(f'Moving average for 7 days of COVID-19 cases in Pará')
plt.plot(casos_media_movel_7dias['Casos_DataSintoma_MM_7dias_PA'], label='Cases_DateSymptom_MM_7days_PA')
plt.plot(casos_media_movel_7dias['Casos_Publicacao_MM_7dias_PA'], label='Cases_Publication_MM_7days_PA')
plt.legend()

print(casos_media_movel_7dias)  

# Salvando o frafico em PNG:
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Moving average for 7 days of COVID-19 cases in Pará - Data collected day {strDiaColetado}.png', dpi=300, transparent=True, bbox_inches='tight')


In [ ]:
#         PARA 14 DIAS - CASOS

# Filtrando apenas a data e retirando o horario:
casos_media_movel_14dias['data'] = pd.to_datetime(casos_media_movel_14dias['data'].dt.strftime('%Y-%m-%d'))

# Tornando Data em indice:
casos_media_movel_14dias = casos_media_movel_14dias.set_index('data')

#Alterando nome das colunas
casos_media_movel_14dias = casos_media_movel_14dias.rename(columns={'Qtd. pela data da publicação':'Casos_Publicacao_MM_14dias_PA','Qtd. pela data do sintoma':'Casos_DataSintoma_MM_14dias_PA'})
                                                                                 

#Plotando os dados:
strDiaColetado = str(casos_media_movel_14dias.index[-1])
strDiaColetado = strDiaColetado.split(" ")[0]

# Plotando os dados:
plt.figure(figsize=(16, 3))
plt.grid()
plt.xlabel('date')
plt.ylabel('cases')

plt.title(f'Moving average for 14 days of COVID-19 cases in Pará')
plt.plot(casos_media_movel_14dias['Casos_DataSintoma_MM_14dias_PA'], label='Cases_DateSymptom_MM_14days_PA')
plt.plot(casos_media_movel_14dias['Casos_Publicacao_MM_14dias_PA'], label='Cases_Publication_MM_14days_PA')
plt.legend()

print(casos_media_movel_14dias)         

# Salvando o frafico em PNG: 
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Moving average for 14 days of COVID-19 cases in Pará - Data collected day {strDiaColetado}.png', dpi=300, transparent=True, bbox_inches='tight')

##Tratamento de vacinação 

In [ ]:
# Normalização e criação de novas colunas em decimais (0=0% e 1=100%)
dados_PA_vacinados['Vacinacao_Dose1_%decimal_PA'] = dados_PA_vacinados ['Vacinacao_Dose1_PA'] * 0.01                                                               
dados_PA_vacinados['Vacinacao_Dose2_%decimal_PA'] = dados_PA_vacinados ['Vacinacao_Dose2_PA'] * 0.01
dados_PA_vacinados['Vacinacao_Dose3_%decimal_PA'] = dados_PA_vacinados ['Vacinacao_Dose3_PA'] * 0.01

# Filtrando os dados em decimais
dados_PA_vacinacao_dec = dados_PA_vacinados.filter(items = ['data',
                                                            'Vacinacao_Dose1_%decimal_PA',
                                                            'Vacinacao_Dose2_%decimal_PA',
                                                            'Vacinacao_Dose3_%decimal_PA'])

#Renomeano o nome  das colunas apenas para o gráfico  
plt.figure(figsize=(16, 4))
plt.grid()
plt.xlabel('date')
plt.ylabel('doses')
plt.title('COVID-19 vaccination graph in Pará (Treaty 0-1)')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose1_%decimal_PA'], label='Vaccination_dose1_PA')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose2_%decimal_PA'], label='Vaccination_dose2_PA')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose3_%decimal_PA'], label='Vaccination_dose3_PA')
plt.legend()

print(dados_PA_vacinacao_dec)   

# Salvando o frafico em PNG: 
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/COVID-19 Vaccination graph in Pará (Treaty 0-1) - Data collected day {date.today()}.png', dpi=300, transparent=True, bbox_inches='tight')  

In [ ]:
# Tornando Data em indice:
dados_PA_vacinacao_dec = dados_PA_vacinacao_dec.set_index('data')
dados_PA_vacinacao_dec
dados_PA_vacinacao_dec

#Integração dos dados

In [ ]:
#Integrando dados e imprimindo os dados
dados_unidos = pd.concat([casos_media_movel_atual,  casos_media_movel_7dias, casos_media_movel_14dias, dados_PA_vacinacao_dec], axis=1)

# obitos_media_movel_atual, obitos_media_movel_7dias, obitos_media_movel_14dias,
dados_unidos

In [ ]:
# Imprimindo o grafico:

# Plotando os dados:
plt.figure(figsize=(20, 8))
plt.grid()
plt.xlabel('date')
plt.ylabel('data')
plt.title(f'Pará Partial Graph - Data collected daily {strDiaColetado}')

#VACINAÇÃO

plt.plot(dados_unidos['Vacinacao_Dose1_%decimal_PA'], label='Vaccination_dose1_PA')
plt.plot(dados_unidos['Vacinacao_Dose2_%decimal_PA'], label='Vaccination_dose2_PA')
plt.plot(dados_unidos['Vacinacao_Dose3_%decimal_PA'], label='Vaccination_dose3_PA')

#CASOS

#Dias atuais
plt.plot(dados_unidos['Casos_DataSintoma_MM_atual_PA'], label='Cases_DateSymptom_MM_Current_PA')
plt.plot(dados_unidos['Casos_Publicacao_MM_atual_PA'], label='Cases_Publication_MM_Current_PA')

#7 dias
plt.plot(dados_unidos['Casos_DataSintoma_MM_7dias_PA'], label='Cases_DateSymptom_MM_7days_PA')
plt.plot(dados_unidos['Casos_Publicacao_MM_7dias_PA'], label='Cases_Publication_MM_7days_PA')

#14 dias
plt.plot(dados_unidos['Casos_DataSintoma_MM_14dias_PA'], label='Cases_DateSymptom_MM_14days_PA')
plt.plot(dados_unidos['Casos_Publicacao_MM_14dias_PA'], label='Cases_Publication_MM_14days_PA')
plt.legend()

# Salvando o frafico em PNG: 
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Pará Partial Graph - Data collected daily {date.today()}.png', dpi=300, transparent=True, bbox_inches='tight')  

#Exportação dos dados

In [ ]:
# Exportando dados em Excel
pasta_exportacao = "/content/drive/MyDrive/NPCA - COVID/_CASOS/DataSet/"
data_exportacao = time.localtime() 
data_exportacao_string = time.strftime("%d_%m_%Y")
dados_unidos.to_excel(pasta_exportacao+'Dataset_'+str(data_exportacao_string)+'.xlsx', index = True)

#**OBITOS**

#Leitura e visualização dos dados

##Óbitos média móvel Atual, 7 Dias, 14 Dias

In [ ]:
#Seleciona os arquivos de OBITOS recentemente baixado da pasta de .CSVs (LakeData)
pasta = '/content/drive/MyDrive/NPCA - COVID/LakeData/Óbitos/' 
lista_de_arquivos_obitos = glob.glob(pasta+'*') 
print(*lista_de_arquivos_obitos, sep = '\n')

In [ ]:
# Indice dos arquivos na lista ordenada. ALTERAR NUMEROS PARA 7 E 14 QUANDO TIVERMOS AO MENOS 15 ARQUIVOS NA PASTA
indice_Setimo_dia_obitos= 7
indice_DecimoQuarto_dia_obitos = 14

# Ordena arquivos por data de criação do arquivo. Ultimo arquivo a ser criado começa no inicio da lista.
lista_de_arquivos_ordenados_obitos = sorted(lista_de_arquivos_obitos,key=os.path.getctime,reverse=True)
print(*lista_de_arquivos_ordenados_obitos,sep = "\n")

In [ ]:
# Seleciona o ultimo arquivo que foi criado e adicioado na pasta
arquivo_atual_obitos = lista_de_arquivos_ordenados_obitos[0]

# Seleciona o setimo arquivo que foi criado depois do ultimo arquivo adicionado na pasta
arquivo_7dias_obitos = lista_de_arquivos_ordenados_obitos[indice_Setimo_dia_obitos]

# Seleciona o decimo quarto arquivo que foi criado depois do ultimo arquivo adicionado na pasta
arquivo_14dias_obitos = lista_de_arquivos_ordenados_obitos[indice_DecimoQuarto_dia_obitos]

print(arquivo_atual_obitos)
print(arquivo_7dias_obitos)
print(arquivo_14dias_obitos)

In [ ]:
# Seleciona o decimo quarto arquivo que foi criado depois do ultimo arquivo adicionado na pasta

#arquivo_14dias_obitos = lista_de_arquivos_ordenados_obitos[20]
#print(arquivo_14dias_obitos)

In [ ]:
#df_teste = pd.read_csv('/content/drive/MyDrive/NPCA - COVID/LakeData/Óbitos/MMO_PA_31_10_2022.csv')
#df_teste

In [ ]:
#Importando arquivos
obitos_media_movel_atual = pd.read_csv(arquivo_atual_obitos, parse_dates=['data'])
obitos_media_movel_atual.info()

obitos_media_movel_7dias = pd.read_csv(arquivo_7dias_obitos, parse_dates=['data'])
obitos_media_movel_7dias.info()

obitos_media_movel_14dias = pd.read_csv(arquivo_14dias_obitos, parse_dates=['data'])
obitos_media_movel_14dias.info()

##Porcentagem de vacinação: primeira, segunda e terceira dose.

In [ ]:
# importando os dados da COVID diretamente do GitHub
dados = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")

# alterar a coluna de data para o formato de data e hora
dados['date'] = pd.to_datetime(dados['date'])

# Filtrando os dados do Braisl, e mostrando apenas as colunas especificas
dados_PA_vacinados = dados.query("state == 'PA'")[['date', 
                                                   'vaccinated_per_100_inhabitants',
                                                   'vaccinated_second_per_100_inhabitants',
                                                   'vaccinated_third_per_100_inhabitants']]

dados_PA_vacinados.columns=['data','Vacinacao_Dose1_PA','Vacinacao_Dose2_PA','Vacinacao_Dose3_PA']

# imprimindo todas as colunas disponiveis
#print(dados.columns)

# Plotando os dados
#dados_PA_vacinados.plot.line(title='Vaccination doses in Pará', x="data", y= ['Vacinacao_Dose1_PA',
#                                                                              'Vacinacao_Dose2_PA',
#     
#                                                                         'Vacinacao_Dose3_PA'], figsize=(16,4)).grid(); 
plt.figure(figsize=(16, 4))
plt.grid()
plt.xlabel('date')
plt.ylabel('doses')
plt.title('COVID-19 Vaccination graph in Pará')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose1_PA'], label='Vaccination_Dose1_PA')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose2_PA'], label='Vaccination_Dose2_PA')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose3_PA'], label='Vaccination_Dose3_PA')
plt.legend()

# Imprimindo o dataframe:
print(dados_PA_vacinados)  

# Salvando o gráfico em PNG:
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Vaccination doses in Pará - Data collected day {date.today()}.png', dpi=300, transparent=True, bbox_inches='tight')                                                                                    

# Tratamento dos dados


##Tratamento para obitos média móvel 

In [ ]:
#         PARA O DIA ATUAL - OBITOS

# Filtrando apenas a data e retirando o horario
obitos_media_movel_atual['data'] = pd.to_datetime(obitos_media_movel_atual['data'].dt.strftime('%Y-%m-%d'))

# Tornando Data em indice:
obitos_media_movel_atual = obitos_media_movel_atual.set_index('data')

#Alterando nome das colunas
obitos_media_movel_atual = obitos_media_movel_atual.rename(columns={"Qtd. pela data do óbito":"Obitos_DataOcorrencia_MM_atual_PA","Qtd. pela data da publicação":"Obitos_Publicacao_MM_atual_PA"}) 
                                                                                               
# Plotando os dados:
strDiaColetado = str(obitos_media_movel_atual.index[-1])
strDiaColetado = strDiaColetado.split(" ")[0]

# Plotando os dados:
plt.figure(figsize=(16, 3))
plt.grid()
plt.xlabel('date')
plt.ylabel('deaths')

plt.title(f'Current day moving average of COVID-19 deaths in Pará')
plt.plot(obitos_media_movel_atual['Obitos_DataOcorrencia_MM_atual_PA'], label='Deaths_DateOccurrence_MM_Current_PA')
plt.plot(obitos_media_movel_atual['Obitos_Publicacao_MM_atual_PA'], label='Deaths_Publication_MM_Current_PA')
plt.legend()

print(obitos_media_movel_atual) 

# Salvando o frafico em PNG: 
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Moving average of deaths from COVID-19 in Pará - Data collected day {strDiaColetado}.png', dpi=300, transparent=True, bbox_inches='tight')


In [ ]:
#         PARA 7 DIAS - OBITOS

# Filtrando apenas a data e retirando o horario
obitos_media_movel_7dias['data'] = pd.to_datetime(obitos_media_movel_7dias['data'].dt.strftime('%Y-%m-%d'))

# Tornando Data em indice:
obitos_media_movel_7dias = obitos_media_movel_7dias.set_index('data')


#Alterando nome das colunas
obitos_media_movel_7dias = obitos_media_movel_7dias.rename(columns={"Qtd. pela data do óbito":"Obitos_DataOcorrencia_MM_7dias_PA","Qtd. pela data da publicação":"Obitos_Publicacao_MM_7dias_PA"})

# Plotando os dados:
strDiaColetado = str(obitos_media_movel_7dias.index[-1])
strDiaColetado = strDiaColetado.split(" ")[0]

# Plotando os dados:
plt.figure(figsize=(16, 3))
plt.grid()
plt.xlabel('date')
plt.ylabel('deaths')

plt.title(f'Moving average for 7 days of COVID-19 deaths in Pará')
plt.plot(obitos_media_movel_7dias['Obitos_DataOcorrencia_MM_7dias_PA'], label='Deaths_DateOccurrence_MM_7days_PA')
plt.plot(obitos_media_movel_7dias['Obitos_Publicacao_MM_7dias_PA'], label='Deaths_Publication_MM_7days_PA')
plt.legend()

print(obitos_media_movel_7dias)

# Salvando o frafico em PNG: 
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Moving average for 7 days of deaths from COVID-19 in Pará - Data collected day {strDiaColetado}.png', dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
#df_teste = pd.read_csv('/content/drive/MyDrive/NPCA - COVID/')
#df_teste

In [ ]:
#         PARA 14 DIAS - OBITOS

# Filtrando apenas a data e retirando o horario
obitos_media_movel_14dias['data'] = pd.to_datetime(obitos_media_movel_14dias['data'].dt.strftime('%Y-%m-%d'))

# Tornando Data em indice:
obitos_media_movel_14dias = obitos_media_movel_14dias.set_index('data')

#Alterando nome das colunas
obitos_media_movel_14dias = obitos_media_movel_14dias.rename(columns={"Qtd. pela data do óbito":"Obitos_DataOcorrencia_MM_14dias_PA","Qtd. pela data da publicação":"Obitos_Publicacao_MM_14dias_PA"})

# Plotando os dados:
strDiaColetado = str(obitos_media_movel_14dias.index[-1])
strDiaColetado = strDiaColetado.split(" ")[0]

# Plotando os dados:
plt.figure(figsize=(16, 3))
plt.grid()
plt.xlabel('date')
plt.ylabel('deaths')

plt.title(f'Moving average for 14 days of COVID-19 cases in Pará')
plt.plot(obitos_media_movel_14dias['Obitos_DataOcorrencia_MM_14dias_PA'], label='Deaths_dateOccurrence_MM_14days_PA')
plt.plot(obitos_media_movel_14dias['Obitos_Publicacao_MM_14dias_PA'], label='Deaths_Publication_MM_14days_PA')
plt.legend()                                                                                                 

print(obitos_media_movel_14dias) 

# Salvando o frafico em PNG: 
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Moving average for 14 days of deaths from COVID-19 in Pará - Data collected day {strDiaColetado}.png', dpi=300, transparent=True, bbox_inches='tight')


##Tratamento de vacinação 


In [ ]:
# Normalização e criação de novas colunas em decimais (0=0% e 1=100%)
dados_PA_vacinados['Vacinacao_Dose1_%decimal_PA'] = dados_PA_vacinados ['Vacinacao_Dose1_PA'] * 0.01                                                                     
dados_PA_vacinados['Vacinacao_Dose2_%decimal_PA'] = dados_PA_vacinados ['Vacinacao_Dose2_PA'] * 0.01
dados_PA_vacinados['Vacinacao_Dose3_%decimal_PA'] = dados_PA_vacinados ['Vacinacao_Dose3_PA'] * 0.01


# Filtrando os dados em decimais
dados_PA_vacinacao_dec = dados_PA_vacinados.filter(items = ['data',
                                                            'Vacinacao_Dose1_%decimal_PA',
                                                            'Vacinacao_Dose2_%decimal_PA',
                                                            'Vacinacao_Dose3_%decimal_PA'])

#Renomeano o nome  das colunas apenas para o gráfico  
plt.figure(figsize=(16, 4))
plt.grid()
plt.xlabel('date')
plt.ylabel('doses')
plt.title('COVID-19 vaccination graph in Pará (Treaty 0-1)')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose1_%decimal_PA'], label='Vaccination_dose1_PA')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose2_%decimal_PA'], label='Vaccination_dose2_PA')
plt.plot(dados_PA_vacinados['data'], dados_PA_vacinados['Vacinacao_Dose3_%decimal_PA'], label='Vaccination_dose3_PA')
plt.legend()

# Imprimindo as DataFrame
print(dados_PA_vacinacao_dec)    

# Salvando o frafico em PNG: 
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/COVID-19 Vaccination graph in Pará (Treaty 0-1) - Data collected day {date.today()}.png', dpi=300, transparent=True, bbox_inches='tight')  

In [ ]:
# Tornando Data em indice:
dados_PA_vacinacao_dec = dados_PA_vacinacao_dec.set_index('data')
dados_PA_vacinacao_dec                                                                                            

#Integração dos dados OBITOS, CASOS E VACINAÇÃO

In [ ]:
#Integrando dados e imprimindo os dados
dados_unidos = pd.concat([obitos_media_movel_atual, obitos_media_movel_7dias, obitos_media_movel_14dias,
                          casos_media_movel_atual,  casos_media_movel_7dias, casos_media_movel_14dias,
                          dados_PA_vacinacao_dec], axis=1)

dados_unidos

In [ ]:
# Imprimindo o grafico:

# Plotando os dados:
plt.figure(figsize=(22, 8))
plt.grid()
plt.xlabel('date')
plt.ylabel('dados')
plt.title(f'Graph of data integration of COVID-19 in Pará - Data collected daily {strDiaColetado}')

#VACINAÇÃO

plt.plot(dados_unidos['Vacinacao_Dose1_%decimal_PA'], label='Vaccination_dose1_PA')
plt.plot(dados_unidos['Vacinacao_Dose2_%decimal_PA'], label='Vaccination_dose2_PA')
plt.plot(dados_unidos['Vacinacao_Dose3_%decimal_PA'], label='Vaccination_dose3_PA')

#CASOS

#Dias atuais
plt.plot(dados_unidos['Casos_DataSintoma_MM_atual_PA'], label='Cases_DateSymptom_MM_Current_PA')
plt.plot(dados_unidos['Casos_Publicacao_MM_atual_PA'], label='Cases_Publication_MM_Current_PA')

#7 dias
plt.plot(dados_unidos['Casos_DataSintoma_MM_7dias_PA'], label='Cases_DateSymptom_MM_7days_PA')
plt.plot(dados_unidos['Casos_Publicacao_MM_7dias_PA'], label='Cases_Publication_MM_7days_PA')

#14 dias
plt.plot(dados_unidos['Casos_DataSintoma_MM_14dias_PA'], label='Cases_DateSymptom_MM_14days_PA')
plt.plot(dados_unidos['Casos_Publicacao_MM_14dias_PA'], label='Cases_Publication_MM_14days_PA')

#OBITOS

#Dia atual
plt.plot(dados_unidos['Obitos_DataOcorrencia_MM_atual_PA'], label='Deaths_dateOccurrence_MM_Current_PA')
plt.plot(dados_unidos['Obitos_Publicacao_MM_atual_PA'], label='Deaths_Publication_MM_Current_PA')

#7 dias
#plt.plot(dados_unidos['Obitos_DataOcorrencia_MM_7dias_PA'], label='Deaths_dateOccurrence_MM_7days_PA')
plt.plot(dados_unidos['Obitos_Publicacao_MM_7dias_PA'], label='Deaths_Publication_MM_7days_PA')

#14 dias
plt.plot(dados_unidos['Obitos_DataOcorrencia_MM_14dias_PA'], label='Deaths_dateOccurrence_MM_14days_PA')
plt.plot(dados_unidos['Obitos_Publicacao_MM_14dias_PA'], label='Deaths_Publication_MM_14days_PA')
plt.legend()
                                                                                                  
# Salvando o frafico em PNG: 
plt.savefig(f'/content/drive/MyDrive/NPCA - COVID/Graficos/Graph of data integration of COVID-19 in Pará - Data collected daily {date.today()}.png', dpi=300, transparent=True, bbox_inches='tight')  

# Exportação dos dados

In [ ]:
# Exportando dados em Excel
pasta_exportacao = "/content/drive/MyDrive/NPCA - COVID/_OBITOS/DataSet/"
data_exportacao = time.localtime() 
data_exportacao_string = time.strftime("%d_%m_%Y")
dados_unidos.to_excel(pasta_exportacao+'Dataset_'+str(data_exportacao_string)+'.xlsx', index = True)

Continua no script de INVESTIGAR OS DADOS.

Link:  https://colab.research.google.com/drive/1_Yk77j6wZjSCwAgtUkbSHgT-xWrdSmL3#scrollTo=NJC8mlZdv95Z